## CB-SDA

In [1]:
import re
import json
import string
import dictances

import numpy as np
import pandas as pd

from sklearn.metrics import ndcg_score
from scipy.stats import wasserstein_distance


In [2]:
users = pd.read_csv('ratings.csv')
dataset = pd.read_csv('fixed.csv')

In [3]:
dataset

,Unnamed: 0,Unnamed: 0.1,adult,genres,id,original_language,original_title,overview,popularity,runtime,title,vote_average,vote_count,actors,director,idx,synopsis
0,0,0,False,"(['animation', 'comedy', 'family'], [0.3333333...",862,en,Toy Story,"led by woody, andy's toys live happily in his ...",21.946943,81.0,Toy Story,7.7,5415.0,"(['tom hanks', ' tim allen', ' don rickles', '...","(['john lasseter'], [1.0])",0,"(['afraid', 'against', 'and', 'andy', 'aside',..."
1,1,1,False,"(['adventure', 'fantasy', 'family'], [0.333333...",8844,en,Jumanji,when siblings judy and peter discover an encha...,17.015539,104.0,Jumanji,6.9,2413.0,"(['robin williams', ' jonathan hyde', ' kirste...","(['joe johnston'], [1.0])",1,"(['26', 'adult', 'alan', 'all', 'an', 'and', '..."
2,2,2,False,"(['romance', 'comedy'], [0.5, 0.5])",15602,en,Grumpier Old Men,a family wedding reignites the ancient feud be...,11.712900,101.0,Grumpier Old Men,6.5,92.0,"(['walter matthau', ' jack lemmon', ' annmargr...","(['howard deutch'], [1.0])",2,"(['alarming', 'ancient', 'and', 'at', 'away', ..."
3,3,3,False,"(['comedy', 'drama', 'romance'], [0.3333333333...",31357,en,Waiting to Exhale,"cheated on, mistreated and stepped on, the wom...",3.859495,127.0,Waiting to Exhale,6.1,34.0,"(['whitney houston', ' angela bassett', ' lore...","(['forest whitaker'], [1.0])",3,"(['all', 'and', 'are', 'bernie', 'better', 'br..."
4,4,4,False,"(['comedy'], [1.0])",11862,en,Father of the Bride Part II,just when george banks has recovered from his ...,8.387519,106.0,Father of the Bride Part II,5.7,173.0,"(['steve martin', ' diane keaton', ' martin sh...","(['charles shyer'], [1.0])",4,"(['and', 'arrival', 'banks', 'both', 'but', 'c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40274,40274,42152,False,"(['horror'], [1.0])",289923,en,The Burkittsville 7,a film archivist revisits the story of rustin ...,0.386450,30.0,The Burkittsville 7,7.0,1.0,"(['monty bane', ' lucy butler', ' david gramme...","(['ben rock'], [1.0])",40274,"(['archivist', 'blair', 'children', 'film', 'h..."
40275,40275,42153,False,"(['science', 'fiction'], [0.5, 0.5])",222848,en,Caged Heat 3000,it's the year 3000 ad. the world's most danger...,0.661558,85.0,Caged Heat 3000,3.5,1.0,"(['lisa boyle', ' kena land', ' zaneta polard'...","(['aaron osborne'], [1.0])",40275,"(['3000', '45', 'accused', 'action', 'ad', 'an..."
40276,40276,42154,False,"(['drama', 'action', 'romance'], [0.3333333333...",30840,en,Robin Hood,"yet another version of the classic epic, with ...",5.683753,104.0,Robin Hood,5.7,26.0,"(['patrick bergin', ' uma thurman', ' david mo...","(['john irvin'], [1.0])",40276,"(['also', 'another', 'are', 'but', 'characters..."
40277,40277,42155,False,"(['drama'], [1.0])",111109,tl,Siglo ng Pagluluwal,an artist struggles to finish his work while a...,0.178241,360.0,Century of Birthing,9.0,3.0,"(['angel aquino', ' perry dizon', ' hazel oren...","(['lav diaz'], [1.0])",40277,"(['about', 'an', 'artist', 'cult', 'finish', '..."


In [4]:
users = users[users.groupby('userId')['userId'].transform('size') > 300]
users

,userId,movieId,rating,timestamp
1710,24,1,4.0,979869938
1711,24,2,3.0,979974023
1712,24,6,4.0,979870499
1713,24,16,3.0,979870379
1714,24,17,3.0,979974163
...,...,...,...,...
26023517,270887,171439,1.0,1500925554
26023518,270887,171755,3.0,1493850948
26023519,270887,173149,5.0,1497402632
26023520,270887,173405,5.0,1496285130


In [5]:
user_profile = users.merge( dataset, left_on='movieId', right_on='id')
user_profile

,userId,movieId,rating,timestamp,Unnamed: 0,Unnamed: 0.1,adult,genres,id,original_language,...,overview,popularity,runtime,title,vote_average,vote_count,actors,director,idx,synopsis
0,24,2,3.0,979974023,4192,4236,False,"(['drama', 'crime'], [0.5, 0.5])",2,fi,...,taisto kasurinen is a finnish coal miner whose...,3.860491,69.0,Ariel,7.1,44.0,"(['turo pajala', ' susanna haavisto', ' matti ...","(['aki kaurismaki'], [1.0])",4192,"(['about', 'and', 'as', 'but', 'coal', 'commit..."
1,49,2,4.5,1149447909,4192,4236,False,"(['drama', 'crime'], [0.5, 0.5])",2,fi,...,taisto kasurinen is a finnish coal miner whose...,3.860491,69.0,Ariel,7.1,44.0,"(['turo pajala', ' susanna haavisto', ' matti ...","(['aki kaurismaki'], [1.0])",4192,"(['about', 'and', 'as', 'but', 'coal', 'commit..."
2,62,2,2.0,1095394231,4192,4236,False,"(['drama', 'crime'], [0.5, 0.5])",2,fi,...,taisto kasurinen is a finnish coal miner whose...,3.860491,69.0,Ariel,7.1,44.0,"(['turo pajala', ' susanna haavisto', ' matti ...","(['aki kaurismaki'], [1.0])",4192,"(['about', 'and', 'as', 'but', 'coal', 'commit..."
3,120,2,2.5,1211653464,4192,4236,False,"(['drama', 'crime'], [0.5, 0.5])",2,fi,...,taisto kasurinen is a finnish coal miner whose...,3.860491,69.0,Ariel,7.1,44.0,"(['turo pajala', ' susanna haavisto', ' matti ...","(['aki kaurismaki'], [1.0])",4192,"(['about', 'and', 'as', 'but', 'coal', 'commit..."
4,229,2,3.0,1037827385,4192,4236,False,"(['drama', 'crime'], [0.5, 0.5])",2,fi,...,taisto kasurinen is a finnish coal miner whose...,3.860491,69.0,Ariel,7.1,44.0,"(['turo pajala', ' susanna haavisto', ' matti ...","(['aki kaurismaki'], [1.0])",4192,"(['about', 'and', 'as', 'but', 'coal', 'commit..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4900366,270123,125249,3.5,1436135273,17370,17744,False,"(['action'], [1.0])",125249,en,...,japanese master spy daka operates a covert esp...,0.979481,260.0,The Bat Man,7.0,11.0,"(['lewis wilson', ' douglas croft', ' j carrol...","(['lambert hillyer'], [1.0])",17370,"(['american', 'covert', 'daka', 'deserted', 'e..."
4900367,270123,140222,3.0,1440920139,23287,23990,False,"(['drama', 'romance', 'war'], [0.3333333333333...",140222,en,...,when a young soldier in vietnam gets dumped by...,6.617821,92.0,Love and Honor,5.8,135.0,"(['liam hemsworth', ' teresa palmer', ' aimee ...","(['danny mooney'], [1.0])",23287,"(['and', 'awol', 'back', 'best', 'by', 'decide..."
4900368,270123,151509,3.0,1453527817,24388,25151,False,"(['comedy', 'music', 'mystery'], [0.3333333333...",151509,en,...,"the long awaited sequel to r. kelly's ""trapped...",0.433321,45.0,Trapped in the Closet: Chapters 23-33,6.5,4.0,"(['r kelly', ' leshay n tomlinson', ' rolando ...","(['r. kelly'], [1.0])",24388,"(['awaited', 'closet', 'in', 'kelly', 'long', ..."
4900369,270123,159109,1.5,1465361347,32492,33689,False,"(['comedy', 'drama', 'horror', 'mystery'], [0....",159109,en,...,"after being released from prison, a man known ...",1.515997,97.0,The Rambler,6.0,14.0,"(['dermot mulroney', ' lindsay pulsipher', ' n...","(['calvin lee reeder'], [1.0])",32492,"(['after', 'and', 'as', 'attempts', 'back', 'b..."


In [6]:
user_profile = user_profile[user_profile.groupby('userId')['userId'].transform('size') > 300]

In [7]:
user_profile.sort_values(by='userId', inplace=True)

<ipython-input-7-4b0a6150f907>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_profile.sort_values(by='userId', inplace=True)


In [8]:
len(user_profile['userId'].unique())

4481

In [9]:
user_profile['rating'] = user_profile['rating'].apply(lambda x: round(x))

<ipython-input-9-c286d0eb2d2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_profile['rating'] = user_profile['rating'].apply(lambda x: round(x))


In [10]:
user_profile.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'Unnamed: 0',
       'Unnamed: 0.1', 'adult', 'genres', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'runtime', 'title',
       'vote_average', 'vote_count', 'actors', 'director', 'idx', 'synopsis'],
      dtype='object')

In [11]:
user_profile.drop(
                    [
                        'timestamp',
                        'adult',
                        'Unnamed: 0',
                        'Unnamed: 0.1',
                        'original_language',
                        'popularity',
                        'vote_count',
                        'runtime'
                    ], axis=1, inplace=True)

C:\Users\Daladier\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
users_for_testing = list(user_profile['userId'].unique())
users_for_testing

[46,
 150,
 229,
 231,
 346,
 390,
 429,
 483,
 533,
 611,
 682,
 741,
 836,
 971,
 1126,
 1136,
 1149,
 1316,
 1652,
 1757,
 1802,
 1846,
 1932,
 2191,
 2313,
 2489,
 2490,
 2520,
 2531,
 2547,
 2555,
 2627,
 2639,
 2749,
 2885,
 2975,
 2981,
 3140,
 3222,
 3437,
 3494,
 3656,
 3673,
 3694,
 3767,
 3771,
 3778,
 3780,
 3792,
 3910,
 3950,
 4073,
 4095,
 4160,
 4219,
 4294,
 4323,
 4351,
 4384,
 4387,
 4450,
 4493,
 4540,
 4666,
 4785,
 4831,
 4856,
 4916,
 4932,
 4937,
 4964,
 5102,
 5229,
 5256,
 5300,
 5332,
 5378,
 5380,
 5472,
 5512,
 5523,
 5620,
 5621,
 5641,
 5662,
 5708,
 5811,
 5829,
 5875,
 5915,
 5953,
 6064,
 6074,
 6109,
 6188,
 6201,
 6222,
 6294,
 6357,
 6448,
 6513,
 6583,
 6659,
 6663,
 6714,
 6785,
 6812,
 6823,
 6914,
 7002,
 7004,
 7040,
 7107,
 7144,
 7155,
 7158,
 7221,
 7238,
 7287,
 7297,
 7386,
 7402,
 7417,
 7437,
 7457,
 7469,
 7515,
 7535,
 7570,
 7849,
 7983,
 8039,
 8215,
 8226,
 8233,
 8273,
 8279,
 8287,
 8317,
 8330,
 8420,
 8482,
 8533,
 8563,
 8622,


### Utility Functions

In [13]:
def stratified_sample_df(df, col, n_samples):
    n = min(n_samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n))
    df_.index = df_.index.droplevel(0)
    return df_

In [14]:
def get_user_profiles(profile_id:int, sample_size: int):
    interest_levels = [0, 1, 2, 3, 4, 5]
    
    profile = user_profile.loc[(user_profile['userId'] == profile_id)]
    train_profile = stratified_sample_df(profile, 'rating', int(sample_size/5))    
    #train_profile = profile.sample(n=sample_size, replace = False, axis=0)
    test_profile = profile.loc[~user_profile['movieId'].isin(train_profile['movieId'].values)]

    test_profile.to_csv(f'profiles/CB/{sample_size}/test_user_{profile_id}.csv')
    
    for interest_level in interest_levels:
        profile = train_profile.loc[train_profile['rating'] == interest_level] 
        symbolic_profile = get_full_profile(profile)

        with open(f'profiles/CB/{sample_size}/profile_{interest_level}_user_{profile_id}_.json', 'w+') as f1:
            json.dump(symbolic_profile, f1)



In [15]:
def get_genres_profile(descriptions: list):
    support = []
    simbolic = []
    for genre_set in descriptions:
        values = fix_variables(genre_set)
        simbolic.append(values)
        support = support + values[0]
    
    support = list(set(support))
    
    results = {}
    for item in support:
        weight_sum = 0
        for simbolic_description in simbolic:
            if item in simbolic_description[0]:
                idx = simbolic_description[0].index(item)
                weight_sum += simbolic_description[1][idx]
        results[item] = weight_sum/len(simbolic)
    
    sorted_results =  sorted(results.items(), key=lambda x: x[1], reverse=True)
    return ([item[0] for item in sorted_results], [item[1] for item in sorted_results])

In [16]:
def fix_variables(items: str):
    items = re.findall(r"\[(.*?)\]", items)
    categories = items[0]
    weights = items[1]
    chars = string.punctuation[:13] + string.punctuation[14:]
    
    categories = ''.join([i for i in categories if i not in string.punctuation])
    weights = ''.join([i for i in weights if i not in chars])
    return categories.split(' '), [float(number) for number in weights.split(' ')]

In [17]:
def fix_variables_actors(items: str):
    items = re.findall(r"\[(.*?)\]", items)
    categories = items[0]
    weights = items[1]
    chars = string.punctuation[:13] + string.punctuation[14:]
    
    new_punctuation = string.punctuation[:11]+ string.punctuation[12:]
    new_punctuation 
    
    categories = ''.join([i for i in categories if i not in new_punctuation])
    weights = ''.join([i for i in weights if i not in chars])
    return categories.split(',  '), [float(number) for number in weights.split(' ')]

In [18]:
def get_actors_profile(descriptions: list):
    support = []
    simbolic = []
    for genre_set in descriptions:
        values = fix_variables_actors(genre_set)
        simbolic.append(values)
        support = support + values[0]
    
    support = list(set(support))
    
    results = {}
    for item in support:
        weight_sum = 0
        for simbolic_description in simbolic:
            if item in simbolic_description[0]:
                idx = simbolic_description[0].index(item)
                weight_sum += simbolic_description[1][idx]
        results[item] = weight_sum/len(simbolic)
    
    sorted_results =  sorted(results.items(), key=lambda x: x[1], reverse=True)
    return ([item[0] for item in sorted_results], [item[1] for item in sorted_results])

In [19]:
def get_directors_profile(descriptions: list):
    support = []
    simbolic = []
    for director in descriptions:
        values = fix_variables_actors(director)
        simbolic.append(values)
        support = support + values[0]
    
    support = list(set(support))
    
    results = {}
    for item in support:
        weight_sum = 0
        for simbolic_description in simbolic:
            if item in simbolic_description[0]:
                idx = simbolic_description[0].index(item)
                weight_sum += simbolic_description[1][idx]
        results[item] = weight_sum/len(simbolic)
    
    sorted_results =  sorted(results.items(), key=lambda x: x[1], reverse=True)
    return ([item[0] for item in sorted_results], [item[1] for item in sorted_results])

In [20]:
def get_synopsis_profile(descriptions: list):
    support = []
    simbolic = []
    for synops in descriptions:
        values = fix_variables(synops)
        simbolic.append(values)
        support = support + values[0]
    
    support = list(set(support))
    
    results = {}
    for item in support:
        weight_sum = 0
        for simbolic_description in simbolic:
            if item in simbolic_description[0]:
                idx = simbolic_description[0].index(item)
                weight_sum += simbolic_description[1][idx]
        results[item] = weight_sum/len(simbolic)
    
    sorted_results =  sorted(results.items(), key=lambda x: x[1], reverse=True)
    return ([item[0] for item in sorted_results], [item[1] for item in sorted_results])

In [21]:
def get_full_profile(dataframe):
    genres = get_genres_profile(dataframe['genres'].values)
    actors = get_actors_profile(dataframe['actors'].values)
    directors = get_directors_profile(dataframe['director'].values)
    synopsis = get_synopsis_profile(dataframe['synopsis'].values)
    
    return {
            'genres':genres,
            'actors': actors,
            'director': directors,
            'synopsis': synopsis
           }

In [22]:
def calculate_alpha(item: tuple, profile:tuple):
    value = 0

    for category in item[0]:
        if category in profile[0]:
            index = item[0].index(category)
            value += item[1][index]
            
    return value

In [23]:
def calculate_beta(item: tuple, profile:tuple):
    value = 0
    for category in item[0]:
        if category in profile[0]:
            index = profile[0].index(category)
            value += profile[1][index]

    return value

In [24]:
def calculate_gamma(item: tuple, profile: tuple):
    value = 0
    intersection = [elem for elem in profile[0] if elem not in item[0]]

    for category in intersection:
        index = profile[0].index(category)
        value += profile[1][index]

    return value

In [25]:
def calculate_delta(item: tuple, profile: tuple):
    value = 0
    intersection = [elem for elem in item[0] if elem not in profile[0]]
    for category in intersection:
        index = item[0].index(category)
        value += item[1][index]

    return value

In [26]:
def sigma_cd(items: tuple, profile:tuple):
    alpha = calculate_alpha(items, profile)
    beta = calculate_beta(items, profile)
    gamma = calculate_gamma(items, profile)
    delta = calculate_delta(items, profile)
    
    value = 0.5 * (((gamma+delta)/(alpha + gamma + delta)) + ((gamma + delta)/(beta + gamma+ delta)))
    return value

In [27]:
def sigma_cf(items: tuple, profile:tuple):

    if len(set(items[0]).intersection(profile[0])) != 0:
        return 0
    union = set(items[0]).union(profile[0])

    score = (len(union) - len(items[0]) - len(profile[0])) / len(union)
    return score

In [28]:
def get_score(sigmas):
    return ((1-sigmas[0]) + sigmas[1])/2

In [29]:
def calculate_sigma(movie: dict, profile: dict, features: list):
    value = 0
    for feature in features:
        value += sigma_cd(movie[feature], profile[feature]) + sigma_cf(movie[feature], profile[feature])
    return value/len(features)

In [30]:
def calculate_dissimilarity(item, user_id, cardinality):
    
    movie = {}
    L = [0, 1, 2, 3, 4, 5]
    rho = [-3, -2, -1, 0, 1, 2]
    
    movie['genres'] = fix_variables(item['genres'])
    movie['actors'] = fix_variables_actors(item['actors'])
    movie['director'] = fix_variables_actors(item['director'])
    movie['synopsis'] = fix_variables(item['synopsis'])
    
    
    score = 0
    for interest_level in L:
        f = open(f'profiles/CB/100/profile_{interest_level}_user_{user_id}_.json')
        profile = json.loads(f.read())
        f.close()
        sigma = calculate_sigma(movie, profile, list(movie.keys()))
        score += (2 * rho[interest_level] * (1 - sigma))/(rho[-1] - rho[0]) 
    return score/len(L)

In [31]:
def get_ndcg(user_id: int, profile_size: int, **kwargs):
    test_items = pd.read_csv(f'profiles/CB/100/test_user_{user_id}.csv', nrows=200)
    test_items['score'] = test_items[['genres', 'actors', 'director', 'synopsis']].apply(calculate_dissimilarity, args=(user_id, profile_size), axis=1)
    
    optimal_rank = list(test_items.sort_values(by='rating', ascending=False)['rating'].values)
    
    estimated_rank = list(test_items.sort_values(by='score', ascending=False)['rating'].values)
    
    k = kwargs.get('k', None)

    return ndcg_score([optimal_rank], [estimated_rank], k=k)

In [33]:
cardialities = [100]
for cardinality in cardialities:

    for user_id in users_for_testing:
        get_user_profiles(user_id, sample_size=cardinality)

In [34]:
ndcgs_20_k10 = []
for idx, user_id in enumerate(users_for_testing[:100]):
    print(f'User {idx} of 100')
    ndcgs_20_k10.append(get_ndcg(user_id=user_id, profile_size=100, k=10))

mean_ndcg_20_k10 = sum(ndcgs_20_k10)/len(ndcgs_20_k10)
mean_ndcg_20_k10

User 0 of 100
User 1 of 100
User 2 of 100
User 3 of 100
User 4 of 100
User 5 of 100
User 6 of 100
User 7 of 100
User 8 of 100
User 9 of 100
User 10 of 100
User 11 of 100
User 12 of 100
User 13 of 100
User 14 of 100
User 15 of 100
User 16 of 100
User 17 of 100
User 18 of 100
User 19 of 100
User 20 of 100
User 21 of 100
User 22 of 100
User 23 of 100
User 24 of 100
User 25 of 100
User 26 of 100
User 27 of 100
User 28 of 100
User 29 of 100
User 30 of 100
User 31 of 100
User 32 of 100
User 33 of 100
User 34 of 100
User 35 of 100
User 36 of 100
User 37 of 100
User 38 of 100
User 39 of 100
User 40 of 100
User 41 of 100
User 42 of 100
User 43 of 100
User 44 of 100
User 45 of 100
User 46 of 100
User 47 of 100
User 48 of 100
User 49 of 100
User 50 of 100
User 51 of 100
User 52 of 100
User 53 of 100
User 54 of 100
User 55 of 100
User 56 of 100
User 57 of 100
User 58 of 100
User 59 of 100
User 60 of 100
User 61 of 100
User 62 of 100
User 63 of 100
User 64 of 100
User 65 of 100
User 66 of 100
User 

0.7073365545763599

In [35]:
cb_data = pd.DataFrame()
cb_data['original_profile_100_200'] = ndcgs_20_k10

In [36]:
cb_data

,original_profile_100_200
0,0.743499
1,0.672727
2,0.621632
3,0.624255
4,0.744444
...,...
95,0.733333
96,0.708571
97,0.710000
98,0.602168


**Método Proposto**

In [41]:
def stratified_sample_df(df, col, n_samples):
    n = min(n_samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n))
    df_.index = df_.index.droplevel(0)
    return df_

In [42]:
def get_user_profiles(profile_id: int, sample_size: int):
    
    profile = user_profile.loc[user_profile['userId'] == profile_id]
    profile = profile.loc[profile['rating'] >= 2]
    train_profile = stratified_sample_df(profile, 'rating', int(sample_size/4))   
    print(train_profile.shape)
    test_profile = profile.loc[~profile.index.isin(train_profile.index.values)]
    test_profile.to_csv(f'profiles/CB/{sample_size}/proposed_method/test_user_{profile_id}.csv')
    
    features = ['genres', 'actors', 'director', 'synopsis']

    profile = {}

    for feature in features: 
      if feature in ['genres', 'synopsis']:
        profile[feature] = get_genres_data(user_id=profile_id, feature_name=feature, train_profile=train_profile)
      else:
        profile[feature] = get_name_data(user_id=profile_id, feature_name=feature, train_profile=train_profile)

    return profile

In [44]:
profiles = {}
for user_id in users_for_testing:
  profiles[user_id] = get_user_profiles(profile_id=user_id, sample_size=100)

(32, 13)
(76, 13)
(100, 13)
(48, 13)
(100, 13)
(88, 13)
(20, 13)
(56, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(8, 13)
(100, 13)
(88, 13)
(72, 13)
(72, 13)
(64, 13)
(52, 13)
(60, 13)
(28, 13)
(100, 13)
(20, 13)
(16, 13)
(100, 13)
(100, 13)
(100, 13)
(8, 13)
(100, 13)
(24, 13)
(24, 13)
(100, 13)
(100, 13)
(4, 13)
(68, 13)
(100, 13)
(100, 13)
(44, 13)
(72, 13)
(64, 13)
(76, 13)
(52, 13)
(80, 13)
(48, 13)
(40, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(40, 13)
(100, 13)
(100, 13)
(56, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(52, 13)
(100, 13)
(40, 13)
(96, 13)
(28, 13)
(68, 13)
(72, 13)
(60, 13)
(100, 13)
(100, 13)
(52, 13)
(20, 13)
(52, 13)
(100, 13)
(100, 13)
(28, 13)
(20, 13)
(100, 13)
(24, 13)
(100, 13)
(8, 13)
(100, 13)
(100, 13)
(88, 13)
(100, 13)
(100, 13)
(48, 13)
(36, 13)
(100, 13)
(44, 13)
(80, 13)
(40, 13)
(64, 13)
(100, 13)
(68, 13)
(92, 13)
(100, 13)
(76, 13)
(100, 13)
(100, 13)
(100, 13)
(60, 13)
(4, 13)
(100, 13)
(60, 13)
(40, 13)
(56, 13)
(100, 13)
(20, 13)
(8, 13

(40, 13)
(40, 13)
(100, 13)
(100, 13)
(12, 13)
(64, 13)
(100, 13)
(40, 13)
(56, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(56, 13)
(60, 13)
(32, 13)
(72, 13)
(12, 13)
(76, 13)
(20, 13)
(68, 13)
(96, 13)
(72, 13)
(32, 13)
(100, 13)
(100, 13)
(100, 13)
(88, 13)
(60, 13)
(68, 13)
(100, 13)
(52, 13)
(100, 13)
(92, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(28, 13)
(100, 13)
(100, 13)
(100, 13)
(56, 13)
(60, 13)
(44, 13)
(100, 13)
(28, 13)
(100, 13)
(96, 13)
(68, 13)
(100, 13)
(12, 13)
(100, 13)
(12, 13)
(32, 13)
(80, 13)
(76, 13)
(56, 13)
(64, 13)
(100, 13)
(56, 13)
(100, 13)
(100, 13)
(16, 13)
(100, 13)
(12, 13)
(100, 13)
(56, 13)
(100, 13)
(88, 13)
(100, 13)
(100, 13)
(75, 13)
(12, 13)
(32, 13)
(40, 13)
(92, 13)
(100, 13)
(92, 13)
(100, 13)
(100, 13)
(8, 13)
(36, 13)
(100, 13)
(100, 13)
(20, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(40, 13)
(100, 13)
(100, 13)
(100, 13)
(56, 13)
(100, 13)
(44, 13)
(44, 13)
(100, 13)
(48, 13)
(100, 13)
(100, 13)
(72, 1

(100, 13)
(4, 13)
(75, 13)
(52, 13)
(16, 13)
(100, 13)
(100, 13)
(20, 13)
(100, 13)
(80, 13)
(12, 13)
(36, 13)
(40, 13)
(100, 13)
(44, 13)
(100, 13)
(100, 13)
(20, 13)
(63, 13)
(32, 13)
(100, 13)
(28, 13)
(20, 13)
(76, 13)
(16, 13)
(100, 13)
(16, 13)
(100, 13)
(24, 13)
(60, 13)
(100, 13)
(60, 13)
(40, 13)
(28, 13)
(80, 13)
(100, 13)
(88, 13)
(100, 13)
(76, 13)
(84, 13)
(100, 13)
(20, 13)
(48, 13)
(100, 13)
(28, 13)
(48, 13)
(20, 13)
(100, 13)
(88, 13)
(60, 13)
(100, 13)
(4, 13)
(24, 13)
(44, 13)
(100, 13)
(96, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(64, 13)
(100, 13)
(100, 13)
(20, 13)
(88, 13)
(100, 13)
(100, 13)
(96, 13)
(75, 13)
(56, 13)
(100, 13)
(100, 13)
(52, 13)
(100, 13)
(88, 13)
(76, 13)
(80, 13)
(100, 13)
(100, 13)
(44, 13)
(100, 13)
(100, 13)
(84, 13)
(100, 13)
(100, 13)
(68, 13)
(100, 13)
(100, 13)
(100, 13)
(8, 13)
(80, 13)
(100, 13)
(40, 13)
(100, 13)
(44, 13)
(48, 13)
(32, 13)
(40, 13)
(20, 13)
(60, 13)
(100, 13)
(88, 13)
(48, 13)
(56, 13)
(100, 13)
(76, 13)
(48, 13

(56, 13)
(68, 13)
(100, 13)
(100, 13)
(68, 13)
(100, 13)
(92, 13)
(64, 13)
(24, 13)
(44, 13)
(100, 13)
(8, 13)
(84, 13)
(52, 13)
(92, 13)
(52, 13)
(100, 13)
(100, 13)
(16, 13)
(100, 13)
(8, 13)
(28, 13)
(32, 13)
(88, 13)
(96, 13)
(100, 13)
(100, 13)
(100, 13)
(64, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(48, 13)
(40, 13)
(100, 13)
(100, 13)
(75, 13)
(100, 13)
(72, 13)
(92, 13)
(12, 13)
(100, 13)
(72, 13)
(80, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(24, 13)
(76, 13)
(100, 13)
(60, 13)
(64, 13)
(100, 13)
(56, 13)
(36, 13)
(100, 13)
(100, 13)
(44, 13)
(32, 13)
(80, 13)
(48, 13)
(20, 13)
(100, 13)
(24, 13)
(75, 13)
(100, 13)
(100, 13)
(56, 13)
(100, 13)
(100, 13)
(40, 13)
(100, 13)
(64, 13)
(44, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(40, 13)
(100, 13)
(100, 13)
(36, 13)
(64, 13)
(100, 13)
(100, 13)
(100, 13)
(12, 13)
(40, 13)
(24, 13)
(100, 13)
(100, 13)
(64, 13)
(100, 13)
(24, 13)
(100, 13)
(100, 13)
(24, 13)
(48, 13)
(72, 1

(100, 13)
(100, 13)
(68, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(56, 13)
(56, 13)
(56, 13)
(84, 13)
(100, 13)
(100, 13)
(16, 13)
(75, 13)
(100, 13)
(16, 13)
(100, 13)
(100, 13)
(52, 13)
(92, 13)
(40, 13)
(12, 13)
(100, 13)
(72, 13)
(48, 13)
(44, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(40, 13)
(100, 13)
(16, 13)
(28, 13)
(100, 13)
(100, 13)
(100, 13)
(36, 13)
(12, 13)
(20, 13)
(76, 13)
(60, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(24, 13)
(100, 13)
(100, 13)
(36, 13)
(100, 13)
(100, 13)
(100, 13)
(40, 13)
(76, 13)
(12, 13)
(28, 13)
(92, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(64, 13)
(16, 13)
(96, 13)
(100, 13)
(24, 13)
(64, 13)
(100, 13)
(100, 13)
(76, 13)
(100, 13)
(60, 13)
(52, 13)
(20, 13)
(75, 13)
(100, 13)
(8, 13)
(100, 13)
(16, 13)
(8, 13)
(100, 13)
(24, 13)
(100, 13)
(100, 13)
(28, 13)
(100, 13)
(8, 13)
(12, 13)
(100, 13)
(4, 13)
(44, 13)
(92, 13)
(100, 13)
(100, 13)
(100, 13)
(64, 13)
(88, 13)
(60, 13)
(100, 13)
(100, 13)
(76, 13

(80, 13)
(100, 13)
(100, 13)
(75, 13)
(20, 13)
(100, 13)
(100, 13)
(52, 13)
(64, 13)
(44, 13)
(100, 13)
(100, 13)
(24, 13)
(100, 13)
(84, 13)
(100, 13)
(84, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(64, 13)
(56, 13)
(68, 13)
(68, 13)
(100, 13)
(36, 13)
(80, 13)
(100, 13)
(56, 13)
(100, 13)
(52, 13)
(100, 13)
(20, 13)
(100, 13)
(100, 13)
(32, 13)
(100, 13)
(100, 13)
(40, 13)
(100, 13)
(100, 13)
(100, 13)
(52, 13)
(100, 13)
(32, 13)
(32, 13)
(40, 13)
(40, 13)
(40, 13)
(60, 13)
(100, 13)
(24, 13)
(100, 13)
(100, 13)
(52, 13)
(68, 13)
(100, 13)
(52, 13)
(32, 13)
(80, 13)
(100, 13)
(36, 13)
(100, 13)
(68, 13)
(24, 13)
(100, 13)
(96, 13)
(100, 13)
(64, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(100, 13)
(52, 13)
(75, 13)
(100, 13)
(36, 13)
(20, 13)
(88, 13)
(100, 13)
(100, 13)
(96, 13)
(100, 13)
(100, 13)
(100, 13)
(84, 13)
(100, 13)
(100, 13)
(64, 13)
(64, 13)
(100, 13)
(64, 13)
(68, 13)
(12, 13)
(100, 13)
(100, 13)
(100, 13)
(68, 13)
(12, 13)
(96, 13)
(100, 13)
(20, 13)
(100

In [39]:
def get_name_data(user_id: int, feature_name: str, train_profile: pd.DataFrame):

  denominator = train_profile['rating'].sum()
  pairs = train_profile[[feature_name, 'rating']].values
  names_dict = {}

  for pair in pairs:
    names = fix_variables_actors(pair[0]) 
    for name in names[0]:
      if name not in names_dict.keys():
        names_dict[name] = names[1][names[0].index(name)] * pair[1]
      else:
        names_dict[name] += names[1][names[0].index(name)] * pair[1] 
  
  names = [key for key in names_dict.keys()]
  weights = [value/denominator for value in names_dict.values()]
  return names, weights

In [40]:
def get_genres_data(user_id: int, feature_name: str,  train_profile: pd.DataFrame):

  denominator = train_profile['rating'].sum()
  pairs = train_profile[[feature_name, 'rating']].values
  genres_dict = {}
  
  for pair in pairs:
    genres = fix_variables(pair[0])
    for genre in genres[0]:
      if genre not in genres_dict.keys():
        genres_dict[genre] = genres[1][genres[0].index(genre)] * pair[1]
      else:
        genres_dict[genre] += genres[1][genres[0].index(genre)] * pair[1] 
  
  genres = [key for key in genres_dict.keys()]
  weights = [value/denominator for value in genres_dict.values()]
  return genres, weights


In [45]:
def get_score(item , user_id: int, profiles: dict):
  movie = {}
  
  movie['genres'] = fix_variables(item['genres'])
  movie['actors'] = fix_variables_actors(item['actors'])
  movie['director'] = fix_variables_actors(item['director'])
  movie['synopsis'] = fix_variables(item['synopsis'])

  score = [] 
  scalling_factors = []
  for feature in movie.keys(): 
    profile_user = profiles[user_id][feature]
    categories = {pair[1]:pair[0] for pair in enumerate(set(profile_user[0]).union(movie[feature][0]))}
    
    scalling_factors.append(len(set(profile_user[0]).union(movie[feature][0])))
    
    u_values = [categories[category] for category in profile_user[0]]
    u_weights = [profile_user[1][profile_user[0].index(category)] for category in profile_user[0]]
    
    v_values = [categories[category] for category in movie[feature][0]]
    v_weights = [movie[feature][1][movie[feature][0].index(category)] for category in movie[feature][0]]
    
    score.append(wasserstein_distance(u_values=u_values, v_values=v_values , u_weights=u_weights, v_weights=v_weights ))

  scalling_factors = np.array([min(scalling_factors)/factor for factor in scalling_factors]) 

  return (np.array(score)*scalling_factors).sum()/len(movie.keys())

In [46]:
def get_score_bhattacharyya(item , user_id: int, profiles: dict):
  movie = {}
  
  movie['genres'] = fix_variables(item['genres'])
  movie['actors'] = fix_variables_actors(item['actors'])
  movie['director'] = fix_variables_actors(item['director'])
  movie['synopsis'] = fix_variables(item['synopsis'])

  score = 0 
  for feature in movie.keys(): 
    profile_user = profiles[user_id][feature]
    profile_user = dict(zip(profile_user[0], profile_user[1]))
    movie_profile = dict(zip(movie[feature][0], movie[feature][1]))
    score += bhattacharyya(profile_user, movie_profile)
    
  return score/len(movie.keys())

In [47]:
def get_score_chi_squared(item , user_id: int, profiles: dict):
  movie = {}
  
  movie['genres'] = fix_variables(item['genres'])
  movie['actors'] = fix_variables_actors(item['actors'])
  movie['director'] = fix_variables_actors(item['director'])
  movie['synopsis'] = fix_variables(item['synopsis'])

  score = 0 
  for feature in movie.keys(): 
    profile_user = profiles[user_id][feature]
    profile_user = dict(zip(profile_user[0], profile_user[1]))
    movie_profile = dict(zip(movie[feature][0], movie[feature][1]))
    score += chi_square(profile_user, movie_profile)
    
  return score/len(movie.keys())

In [48]:
def get_score_mae(item , user_id: int, profiles: dict):
  movie = {}
  
  movie['genres'] = fix_variables(item['genres'])
  movie['actors'] = fix_variables_actors(item['actors'])
  movie['director'] = fix_variables_actors(item['director'])
  movie['synopsis'] = fix_variables(item['synopsis'])

  score = 0 
  for feature in movie.keys(): 
    profile_user = profiles[user_id][feature]
    profile_user = dict(zip(profile_user[0], profile_user[1]))
    movie_profile = dict(zip(movie[feature][0], movie[feature][1]))
    score += dictances.chi_square(profile_user, movie_profile)
    
  return score/len(movie.keys())

In [49]:
def score(movie_id: int, user_id: int, ids: list, full_matrix: np.array, merged: pd.DataFrame):
  ru = merged['rating'].loc[merged['userId'] == user_id].mean()
  
  indexes = full_matrix[ids.index(user_id), :].argsort()[::-1][:30]
  similarities = full_matrix[ids.index(user_id) ,indexes]

  neighbors = [ids[idx] for idx in indexes]
  
  terms = []

  for neighbor in neighbors:
    rv = merged['rating'].loc[merged['userId'] == neighbor].mean()
    aux = merged.loc[(merged['userId']==neighbor)].set_index('movieId')
    rvi = aux['rating'].to_dict().get(movie_id, 0)

    terms.append(rvi - rv)

  return ru + ((np.array(terms) * similarities).sum())/(similarities.sum())

In [50]:
def get_ndcg(user_id: int, profile_size: int, **kwargs):
  test_items = pd.read_csv(f'profiles/CB/{profile_size}/proposed_method/test_user_{user_id}.csv', nrows=200)

  test_items['score'] = test_items[['genres', 'actors', 'director', 'synopsis']].apply(get_score, args= (user_id, profiles), axis=1)

  optimal_rank = list(test_items.sort_values(by='rating', ascending=False)['rating'].values)
   
  estimated_rank = list(test_items.sort_values(by='score', ascending=True)['rating'].values)

  k = kwargs.get('k', None)

  return ndcg_score([optimal_rank], [estimated_rank], k=k)

In [51]:
ndcgs_20_k10 = []
for idx, user_id in enumerate(users_for_testing[:100]):
    print(f'User {idx} of 100')
    ndcgs_20_k10.append(get_ndcg(user_id=user_id, profile_size=100, k=10))

mean_ndcg_20_k10 = sum(ndcgs_20_k10)/len(ndcgs_20_k10)
mean_ndcg_20_k10

User 0 of 100
User 1 of 100
User 2 of 100
User 3 of 100
User 4 of 100
User 5 of 100
User 6 of 100
User 7 of 100
User 8 of 100
User 9 of 100
User 10 of 100
User 11 of 100
User 12 of 100
User 13 of 100
User 14 of 100
User 15 of 100
User 16 of 100
User 17 of 100
User 18 of 100
User 19 of 100
User 20 of 100
User 21 of 100
User 22 of 100
User 23 of 100
User 24 of 100
User 25 of 100
User 26 of 100
User 27 of 100
User 28 of 100
User 29 of 100
User 30 of 100
User 31 of 100
User 32 of 100
User 33 of 100
User 34 of 100
User 35 of 100
User 36 of 100
User 37 of 100
User 38 of 100
User 39 of 100
User 40 of 100
User 41 of 100
User 42 of 100
User 43 of 100
User 44 of 100
User 45 of 100
User 46 of 100
User 47 of 100
User 48 of 100
User 49 of 100
User 50 of 100
User 51 of 100
User 52 of 100
User 53 of 100
User 54 of 100
User 55 of 100
User 56 of 100
User 57 of 100
User 58 of 100
User 59 of 100
User 60 of 100
User 61 of 100
User 62 of 100
User 63 of 100
User 64 of 100
User 65 of 100
User 66 of 100
User 

0.7832100494865606

In [52]:
cb_data['proposed_100_wo_irrelevants'] = ndcgs_20_k10

In [53]:
from scipy.stats import wilcoxon
statistic, p_value = wilcoxon(cb_data['original_profile_100_200'], cb_data['proposed_100_wo_irrelevants'])

if p_value <= 0.05:
  print('Valores diferem')
else:
  print('Valores estatisticamente equivalentes')

Valores diferem


In [54]:
cb_data.mean()

original_profile_100_200       0.707337
proposed_100_wo_irrelevants    0.783210
dtype: float64

In [55]:
cb_data.to_csv('cb_data_100.csv')

In [56]:
cb_data

,original_profile_100_200,proposed_100_wo_irrelevants
0,0.743499,0.830097
1,0.672727,0.736486
2,0.621632,0.579034
3,0.624255,0.798193
4,0.744444,0.738462
...,...,...
95,0.733333,0.731818
96,0.708571,0.755000
97,0.710000,0.729412
98,0.602168,0.837264


In [ ]:
#user
uv=[4, 2, 11, 0, 5, 3, 13, 9, 12, 6, 10, 1, 7, 8]
uw=[0.1619047619047619, 0.31857142857142856, 0.07857142857142857, 0.10428571428571429, 0.037142857142857144, 0.05142857142857142, 0.030476190476190476, 0.030476190476190476, 0.0757142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.011428571428571429, 0.014285714285714285]
#filme
vv=[5, 3, 12]
vw=[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

wasserstein_distance(u_values=uv, v_values=vv , u_weights=uw, v_weights=vw )

In [ ]:
#user
uv=[4, 2, 11, 0, 5, 3, 13, 9, 12, 6, 10, 1, 7, 8]
uw=[0.1619047619047619, 0.31857142857142856, 0.07857142857142857, 0.10428571428571429, 0.037142857142857144, 0.05142857142857142, 0.030476190476190476, 0.030476190476190476, 0.0757142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.011428571428571429, 0.014285714285714285]
#filme
vv=[4, 2, 11, 0, 5, 3, 13, 9, 12, 6, 10, 1, 7, 8]
vw=[0, 0, 0, 0, 0.3333333333333333, 0.3333333333333333, 0, 0, 0.3333333333333333, 0, 0, 0 , 0 ,0]

wasserstein_distance(u_values=uv, v_values=vv , u_weights=uw, v_weights=vw )

In [ ]:
#user
uv=[4, 2, 11, 0, 5, 3, 13, 9, 12, 6, 10, 1, 7, 8]
uw=[0.1619047619047619, 0.31857142857142856, 0.07857142857142857, 0.10428571428571429, 0.037142857142857144, 0.05142857142857142, 0.030476190476190476, 0.030476190476190476, 0.0757142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.011428571428571429, 0.014285714285714285]
#filme
vv=[5, 3, 12, 4, 2, 11, 0, 13, 9, 6, 10, 1, 7, 8]
vw=[0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0 ,0, 0 ,0 ,0]

wasserstein_distance(u_values=uv, v_values=vv , u_weights=uw, v_weights=vw )

In [ ]:
uv=[447, 63, 92, 44, 384, 220, 421, 297, 354, 448, 343, 430, 187, 12, 379, 33, 424, 394, 412, 31, 299, 23, 355, 259, 291, 452, 50, 114, 6, 426, 18, 91, 20, 363, 161, 221, 313, 249, 437, 120, 311, 278, 356, 263, 286, 51, 456, 103, 67, 387, 325, 42, 289, 199, 61, 461, 258, 88, 429, 262, 132, 349, 422, 235, 159, 449, 403, 178, 135, 102, 101, 255, 219, 146, 280, 377, 380, 257, 348, 46, 218, 323, 396, 127, 333, 113, 204, 408, 233, 171, 231, 295, 352, 335, 292, 25, 416, 115, 460, 250, 331, 2, 164, 205, 210, 457, 279, 440, 246, 98, 267, 244, 47, 338, 217, 362, 143, 16, 39, 95, 360, 357, 321, 453, 414, 198, 188, 172, 268, 324, 196, 445, 153, 361, 21, 400, 404, 417, 17, 341, 73, 337, 185, 332, 433, 365, 27, 40, 242, 423, 123, 105, 111, 368, 110, 216, 116, 182, 28, 56, 30, 441, 37, 155, 382, 369, 65, 252, 3, 151, 374, 409, 418, 100, 121, 158, 140, 307, 145, 94, 273, 300, 237, 142, 420, 243, 147, 367, 176, 455, 128, 442, 364, 107, 36, 371, 97, 81, 11, 79, 315, 209, 10, 316, 192, 32, 241, 261, 266, 80, 49, 353, 52, 69, 401, 125, 71, 312, 317, 83, 276, 234, 87, 26, 256, 13, 305, 58, 346, 85, 162, 183, 326, 304, 350, 386, 227, 167, 281, 283, 165, 109, 134, 245, 306, 314, 388, 344, 322, 130, 64, 200, 34, 179, 177, 77, 398, 104, 124, 55, 160, 82, 427, 247, 19, 436, 48, 45, 131, 397, 431, 358, 150, 410, 137, 197, 141, 76, 376, 450, 318, 419, 462, 119, 330, 451, 366, 144, 59, 152, 290, 60, 9, 86, 285, 277, 228, 347, 181, 351, 301, 393, 156, 126, 163, 84, 15, 260, 139, 117, 293, 173, 407, 435, 294, 383, 238, 370, 168, 302, 236, 287, 308, 186, 392, 319, 208, 195, 309, 443, 189, 70, 229, 226, 320, 74, 78, 298, 434, 372, 118, 378, 72, 112, 190, 265, 271, 35, 129, 296, 405, 180, 43, 232, 174, 14, 381, 310, 38, 339, 329, 399, 22, 5, 446, 213, 170, 373, 275, 411, 66, 57, 149, 154, 1, 211, 391, 108, 251, 222, 41, 148, 54, 184, 413, 439, 248, 389, 0, 62, 253, 345, 224, 274, 206, 284, 136, 342, 24, 93, 8, 340, 288, 75, 375, 212, 157, 214, 207, 193, 106, 240, 166, 99, 215, 303, 415, 254, 282, 133, 458, 438, 223, 202, 122, 29, 269, 272, 428, 454, 406, 169, 175, 239, 328, 201, 336, 203, 7, 334, 138, 68, 53, 395, 230, 425, 359, 327]
uw=[0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.002040816326530612, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.003444227005870842, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0021176963936259435, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.0005870841487279843, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.0034529950521107484, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.002484472049689441, 0.005341614906832298, 0.002484472049689441, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.002673469387755102, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.003663865546218487, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.001142857142857143, 0.019047619047619046, 0.019047619047619046, 0.019047619047619046, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0035714285714285713, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0031746031746031746, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.0025210084033613443, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.004081632653061224, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.001530612244897959, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.0009685230024213075, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.003007518796992481, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0027210884353741495, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.0017142857142857142, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.006349206349206349, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.002857142857142857, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286, 0.0026785714285714286]
vv=[390, 96, 402, 191, 385, 89, 270, 432, 90, 459, 444, 264, 225, 4, 194]
vw=[0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667]

wasserstein_distance(u_values=uv, v_values=vv , u_weights=uw, v_weights=vw )

In [ ]:
a = {
    'horror': 0.5,
    'action': 0.5
}

b = {
    'horror': 0.5,
    'action': 0.5
}

In [ ]:
bhattacharyya(a, b)

In [ ]:
mae(a, b)